# Optimisations des objectifs

Ce bloc-notes démontre un cas d'utilisation pour l'optimisation des objectifs d'Amazon Personalize. Souvent, d'autres facteurs influencent les recommandations et l'optimisation des objectifs peut être utilisée pour fournir des données supplémentaires pour gérer le modèle.

Cet exemple suppose que nous fournissons des recommandations pour un service d'abonnement de vidéo à la demande (VOD) en continu. Le service permet aux utilisateurs un accès illimité au catalogue vidéo, mais les titres disponibles dans leur catalogue sont soumis à des conditions de licence différentes selon l'accord avec le propriétaire du contenu. En fournissant des recommandations qui pondèrent le contenu dont les coûts de licence sont les plus bas, le service de streaming peut réduire les coûts de licence tout en gardant les clients satisfaits.

Encore une fois, les données proviennent du projet [MovieLens](https://movielens.org/). Vous pouvez obtenir plus d'informations sur les données et les utilisations potentielles en effectuant une recherche sur Internet pendant l'une des périodes d'attente dans les cellules ci-dessous. Vous ajouterez aux données de MovieLens un champ supplémentaire pour les redevances et générerez les données qui fourniront les coûts des redevances au jeu de données. 

## Comment utiliser le bloc-notes

Le code est décomposé en cellules comme celle ci-dessous. Il y a un bouton `Run` triangulaire en haut de cette page qui vous permet d'exécuter chaque cellule et de passer à la suivante, ou vous pouvez appuyer sur `Shift` + `Enter` lorsque vous êtes dans la cellule pour l'exécuter et passer à la suivante.

Lorsqu'une cellule est en cours d'exécution, vous remarquerez une ligne sur le coté affichant une `*` pendant que la cellule est opérationnelle ou elle se mettra à jour sous forme de nombre pour indiquer la dernière cellule qui a terminé l'exécution après avoir achevé l'exécution de tous les codes dans une cellule.


Suivez les instructions ci-dessous et exécutez les cellules pour démarrer avec l'optimisation des objectifs d'Amazon Personalize.

## Importations 

Python est livré avec une large gamme de bibliothèques. Nous devons les importer, ainsi que celles qui sont installées pour nous aider, comme [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK pour python) et [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), qui sont des outils essentiels pour la science des données.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError

!conda install -y -c conda-forge unzip

pd.options.mode.chained_assignment = None

Ensuite, vous devez vérifier que votre environnement peut communiquer avec Amazon Personalize. Les lignes ci-dessous servent à cela.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configurer les données

Les données sont importées dans Amazon Personalize par le biais d'Amazon S3. Nous allons spécifier ci-dessous les noms de fichiers que vous utilisez pour écrire les fichiers localement avant de les charger sur S3.


In [ ]:
filename = "movie-lens-100k.csv"
items_filename = "movie-lens-items.csv"

### Télécharger, préparer et charger les données d'entraînement

Actuellement, vous n'avez pas encore chargé les données de MovieLens localement pour les examiner. Exécutez les lignes ci-dessous pour télécharger la dernière copie et l'examiner rapidement.

#### Télécharger et explorer le jeu de données

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### Préparer et charger les données

Le code ci-dessous charge les informations sur le film à partir du téléchargement de MovieLens. Il fournit des informations sur le titre, la date de sortie, le lien IMDB et la liste des colonnes indiquant les genres qui s'appliquent au titre.

Il n'y a pas de champ de redevance puisque l'atelier génère une redevance fictive, ce qui sera fait à l'étape suivante.








In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|',encoding='latin-1', names=['ITEM_ID', 'TITLE', 'RELEASE_DATE', 'VIDEO_RELEASE_DATE', 'IMDB_URL', 'MISC', 'ACTION_GENRE', 'ADVENTURE_GENRE', 'ANIMATION_GENRE', 'CHILDRENS_GENRE','COMEDY_GENRE', 'CRIME_GENRE', 'DOCUMENTARY_GENRE','DRAMA_GENRE','FANTASY_GENRE', 'FILMNOIR_GENRE','HORROR_GENRE', 'MUSICAL_GENRE','MYSTERY_GENRE', 'ROMANCE_GENRE', 'SCIFI_GENRE', 'THRILLER_GENRE', 'WAR_GENRE', 'WESTERN_GENRE'     ])
items

#### Ajouter les données de redevance et les genres
Nous attribuons une valeur au champ REDEVANCE avec une distribution régulière des valeurs 0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.10. Cela signifie que la plupart des films ont une redevance relativement faible ou nulle et qu'un petit nombre de films ont une redevance plus élevée. 

Remarquez sur le graphique à barres ci-dessous la distribution régulière des titres avec chaque valeur de redevance.

Le résultat du champ GENRE de MovieLens ci-dessus est catégorique, chaque colonne indiquant si le titre appartient au genre, mais les articles chargés dans la personnalisation peuvent accepter des tableaux. Quelques codes de pandas pour rassembler les genres dans une colonne délimitée par des pipes listant tous les genres.



In [ ]:
pd.set_option('display.max_rows', 10)

royaltyvalues = [0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.10]
items.loc[:,'ROYALTY'] = items['ITEM_ID'].map(lambda x: royaltyvalues[x%8])

items.loc[:,'GENRE']='' 

for col_name in items.columns:
    if col_name.endswith('_GENRE'):
        items.loc[items[col_name]==1,'GENRE']= items['GENRE']+'|'+ col_name[:-6]

items = items[['ITEM_ID', 'TITLE','ROYALTY', 'GENRE']]
items.loc[:,'GENRE'] = items['GENRE'].str[1:]


items.loc[:,'ROYALTY'].value_counts().plot.bar()
items.head(10)


#### Réglez la valeur de la redevance sur un nombre négatif.

L'optimisation d'objectif optimisera les valeurs les plus élevées d'un champ. Dans ce cas, nous devons équilibrer la pertinence du film par rapport aux frais de redevance qui seront encourus par le service de streaming.

Les films dont les redevances sont les plus faibles doivent avoir les valeurs numériques les plus élevées afin de les stimuler. Dans ce cas, nous allons convertir les redevances en un nombre négatif. Pour ce faire, il faut multiplier -1 par la valeur absolue de la redevance.

Remarque : La valeur absolue est utilisée afin de pouvoir exécuter cette cellule deux fois avec les mêmes résultats, ce qui est utile dans un environnement d'atelier.

In [ ]:
items.loc[:,'ROYALTY'] = -1 * abs(items['ROYALTY'])
items

## Configurer un compartiment S3 et un rôle IAM <a class="anchor" id="bucket_role"></a>
Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Utilisez les métadonnées stockées sur l'instance sous-jacente de ce bloc-notes Amazon SageMaker pour déterminer la région dans laquelle il fonctionne. Si vous utilisez un bloc-notes Jupyter en dehors d'Amazon SageMaker, définissez simplement la région sous forme de chaîne comme suit. Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    notebook_data = json.load(notebook_info)
    resource_arn = notebook_data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Les noms des compartiments Amazon S3 sont uniques au niveau mondial. Pour créer un nom de compartiment unique, le code ci-dessous ajoutera la chaîne de caractères `personalize-objective-optimization-` à votre numéro de compte AWS. Puis il crée un compartiment avec ce nom dans la région découverte dans la cellule précédente.

In [ ]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-objective-optimization-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### Charger les données vers S3

Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'article. 

In [ ]:
items = items[['ITEM_ID', 'TITLE', 'GENRE', 'ROYALTY']]
items_dataset = items[['ITEM_ID','ROYALTY', 'GENRE']]

items_dataset.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

### Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

#### Créer le schéma de l'article du titre du film
Nous aurons besoin de deux schémas, un pour les titres de films, qui sera de type article, et un second qui définira la structure des interactions.

In [ ]:
schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "ROYALTY",
            "type": "float"
        },        {
            "name": "GENRE",
            "type": [
                "null",
                "string"
              ],
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_item_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-item-schema"+suffix,
    schema = json.dumps(schema)
)

item_schema_arn = create_item_schema_response['schemaArn']
print(json.dumps(create_item_schema_response, indent=2))

À partir de ce point, nous suivrons le même processus que pour la [première campagne](https://github.com/aws-samples/amazon-personalize-samples/blob/master/getting_started/notebooks/1.Building_Your_First_Campaign.ipynb), en chargeant les interactions. Nous ne devons inclure que les interactions ayant au moins une note de trois, et nous ne voulons pas recommander des films que les spectateurs n'aimeront pas.

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

### Créer un schéma d'interaction

L'un des principaux articles permettant à Personalize de comprendre vos données provient du schéma défini ci-dessous. Cette configuration indique au service comment traiter les données fournies par votre fichier CSV. Notez que les colonnes et les types s'alignent sur les articles présents dans le fichier que vous avez créé ci-dessus.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-schema"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### Créer et attendre un groupe de jeux de données

Le groupe le plus important dans Personalize est le groupe de jeu de données, qui permet d'isoler vos données, vos traceurs d'événements, vos solutions et vos campagnes. Regroupement de choses qui partagent un ensemble commun de données. N'hésitez pas à modifier le nom ci-dessous si vous le souhaitez.

#### Créer un groupe de jeux de données

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-objective-optmization-demo-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Attendre que le groupe de jeu de données ait le statut ACTIF

Avant de pouvoir utiliser le groupe de jeu de données dans les articles ci-dessous, il doit être actif; exécutez la cellule ci-dessous et attendez qu'il soit actif.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Créer un jeu de données

Après le groupe, la prochaine chose à créer est les jeux de données réels. Dans cet exemple, nous allons créer un jeu de données pour les interactions et un autre pour les données des articles. Exécutez les cellules ci-dessous pour le créer.

In [ ]:
def create_dataset(dataset_type, schema_arn, name):
    create_dataset_response = personalize.create_dataset(
        name = name,
        datasetType = dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = schema_arn
    )
    dataset_arn = create_dataset_response['datasetArn']

    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = dataset_arn
        )
        status = describe_dataset_response["dataset"]["status"]
        print("Dataset: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(10)
    return dataset_arn


In [ ]:
interaction_dataset_arn = create_dataset("INTERACTIONS", schema_arn, 'personalize-objective-optmization-interactions-'+suffix)


print('interaction_dataset_arn: ' + interaction_dataset_arn)


In [ ]:
item_dataset_arn = create_dataset("ITEMS", item_schema_arn, 'personalize-objective-optmization-items-'+suffix)
print('item_dataset_arn: ' + item_dataset_arn)

#### Créer un rôle Personalize

De plus, Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin d'avoir les autorisations d'exécuter certaines tâches. Les lignes ci-dessous les accordent.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importer les données

Vous avez précédemment créé le groupe de jeu de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données de S3 dans Amazon Personalize pour les utiliser dans la création de votre modèle. Nous allons charger plusieurs importations. La fonction ci-dessous sera donc utilisée pour démarrer la tâche d'importation, puis surveiller l'achèvement de la tâche d'importation.

#### Créer une tâche d'importation de jeu de données

In [ ]:
def create_dataset_import_job(dataset_arn, dataLocation, name):
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = name,
        datasetArn = dataset_arn,
        dataSource = {
            "dataLocation": dataLocation
        },
        roleArn = role_arn
    )

    dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dataset_import_job_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        print("DatasetImportJob: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)
    return dataset_import_job_arn

#### Charger les interactions
La tâche d'importation ci-dessous chargera le jeu de données d'interaction.

In [ ]:
dataset_import_job_arn = create_dataset_import_job(interaction_dataset_arn, "s3://{}/{}".format(bucket_name, filename), "personalize-objective-optimization-interaction-"+suffix)
print('dataset_import_job_arn: ' + dataset_import_job_arn)

#### Chargez les titres de films dans le jeu de données des articles

La tâche d'importation peut durer un certain temps ; veuillez patienter jusqu'à ce que vous voyiez qu'elle est active ci-dessous.

In [ ]:
item_dataset_import_job_arn = create_dataset_import_job(item_dataset_arn, "s3://{}/{}".format(bucket_name, items_filename), "personalize-objective-optimization-item-"+suffix)
print('item_dataset_import_job_arn: ' + item_dataset_import_job_arn)

## Créer la solution et la version

Dans Amazon Personalize, un modèle entraîné est appelé Solution. Chaque Solution peut avoir plusieurs versions spécifiques qui se rapportent à un volume de données donné lorsque le modèle a été formé.

Pour commencer, nous allons énumérer toutes les recettes qui sont prises en charge. Une recette est un algorithme qui n'a pas encore été entraîné sur vos données. Après l'énumération, vous en choisirez une et l'utiliserez pour créer votre modèle.

### Sélectionner une recette

#### Personnalisation de l'utilisateur
La recette [Personnalisation utilisateur](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) (aws-user-personalization) est optimisée pour tous les scénarios de recommandation USER_PERSONALIZATION. Lorsque vous recommandez des articles, elle utilise l'exploration automatique des articles.

Avec l'exploration automatique, Amazon Personalize teste automatiquement les différentes recommandations d'articles, découvre comment les utilisateurs interagissent avec ces articles recommandés et stimule les recommandations pour les articles qui favorisent un meilleur engagement et une meilleure conversion. Cela améliore la découverte des articles et l'engagement lorsque vous disposez d'un catalogue qui évolue rapidement, ou lorsque de nouveaux articles, tels que des articles d'actualité ou des promotions, sont plus pertinents pour les utilisateurs lorsqu'ils sont frais.

Vous pouvez équilibrer le niveau d'exploration (où les articles présentant moins de données d'interaction ou de pertinence sont recommandés plus fréquemment) et le niveau d'exploitation (où les recommandations sont basées sur ce que nous savons ou sur la pertinence). Amazon Personalize ajuste automatiquement les recommandations futures en fonction des retours implicites des utilisateurs.

Tout d'abord, sélectionnez la recette en trouvant l'ARN dans la liste des recettes ci-dessus.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### Créer et attendre la solution

Vous créez d'abord la solution avec l'API, puis vous créez une version. L'entraînement de votre modèle pour créer votre version de la solution prendra plusieurs minutes. Une fois qu'il a commencé et que vous voyez les notifications en cours, c'est le moment de faire une pause, de prendre un café, etc.

La fonction accepte le paramètre de sensibilité de l'objectif, qui peut être OFF, LOW, MED ou HIGH (INACTIF, FAIBLE, MOYEN ou ÉLEVÉ). Cela permettra d'ajuster la pondération qui introduit l'impact de l'objectif dans le modèle.

La fonction crée la solution et la version initiale de la solution pour cette solution.

#### Créer une solution

In [ ]:
def create_solution(name, objectiveSensitivity):
    create_solution_response = personalize.create_solution(
        name = name,
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn,
        solutionConfig = {
            "optimizationObjective": {
                "itemAttribute": "ROYALTY",
                "objectiveSensitivity":objectiveSensitivity
            }
        }
    )
    
    solution_arn = create_solution_response['solutionArn']
    
    print('solutionArn:' + solution_arn)

    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn
    )

    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print('solution_version_arn: ' + solution_version_arn)

    return {
        "solution_arn": solution_arn,
        "solution_version_arn": solution_version_arn
    }
    
def waitForSolutionVersion(solution_version_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response["solutionVersion"]["status"]
        print("SolutionVersion: {} {}".format(solution_version_arn, status))

        if status == "ACTIVE" or status == "CREATE FAILED"  or status == "CREATE STOPPING":
            break

        time.sleep(60)

#### Créer les versions de la solution
Créer trois solutions différentes, l'une avec l'optimisation des objectifs désactivée, l'une avec un réglage faible et l'autre avec un réglage élevé. Cela lancera trois solutions parallèlement et l'étape suivante attendra qu'elles soient terminées.

In [ ]:
high_solution = create_solution('movie-recommendation-low-royalties-'+suffix, 'HIGH')
low_solution = create_solution('movie-recommendation-medium-royalties-'+suffix, 'LOW')
no_objective_optimization_solution = create_solution('movie-recommendation-max-relevance-'+suffix, 'OFF')

#### Attendre que les versions de la solution aient le statut ACTIF

Cela prendra environ 40 à 50 minutes.

In [ ]:
waitForSolutionVersion(low_solution['solution_version_arn'])
waitForSolutionVersion(no_objective_optimization_solution['solution_version_arn'])
waitForSolutionVersion(high_solution['solution_version_arn'])

#### Obtenir des métriques des versions de la solution

Maintenant que votre solution et sa version sont en place, vous pouvez obtenir les métriques qui vous permettront de juger de ses performances. Ces métriques ne sont pas particulièrement bonnes, car il s'agit d'un jeu de données de démonstration. Cependant, vous vous devriez constater des améliorations avec des jeux de données plus grands et plus complexes.

Vous pouvez constater des différences dans la qualité du modèle en fonction de l'impact des optimisations des objectifs.

In [ ]:
def get_solution_metrics(solutions):
    metricdata = { "name": []}
    
    for key in solutions:
        solution = solutions[key]
        
        metricdata["name"].append(key)
        
        get_solution_metrics_response = personalize.get_solution_metrics(
            solutionVersionArn = solution['solution_version_arn']
        )

        for metricname in get_solution_metrics_response['metrics']:
            if not metricname in metricdata:
                metricdata[metricname] = []
                
            metricdata[metricname].append( get_solution_metrics_response['metrics'][metricname])
            
        # print(json.dumps(get_solution_metrics_response, indent=2))
    return pd.DataFrame.from_dict(metricdata);

metrics = get_solution_metrics({
    "no-optimization": no_objective_optimization_solution,
    "low-optimization": low_solution,
    "high-optimization": high_solution,
})

metrics

Nous recommandons de lire [la documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) pour comprendre les métriques, mais nous avons également copié des parties de la documentation ci-dessous pour plus de commodité.

Vous devez comprendre les termes suivants à propos de l'évaluation dans Personalize :

- *Relevant recommendation* est une recommandation qui correspond à une valeur dans les données de test pour un utilisateur particulier.
- *Rank* fait référence à la position d'un article recommandé dans la liste des recommandations. La position 1 (haut de la liste) est présumée être la plus pertinente pour l'utilisateur.
- *Query* fait référence à l'équivalent interne d'un appel GetRecommendations.

Les métriques fournies par Personalize sont les suivantes :

- coverage : La proportion d'articles uniques recommandés à partir de toutes les requêtes sur le nombre total d'articles uniques dans les données de formation (comprend à la fois les jeux de données d'articles et d'interactions).
- mean_reciprocal_rank_at_25 : La [moyenne des rangs réciproques](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la première recommandation pertinente parmi les 25 premières recommandations pour toutes les requêtes. Cette métrique est appropriée si vous êtes intéressé par la recommandation unique la mieux classée.
- normalized_discounted_cumulative_gain_at_K : Le gain actualisé suppose que les recommandations situées plus bas dans une liste de recommandations sont moins pertinentes que les recommandations situées plus haut. Par conséquent, chaque recommandation est réduite (pondération plus faible) par un facteur dépendant de sa position. Pour produire le [gain cumulé actualisé](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) à K, toutes les recommandations actualisées pertinentes dans les K premières recommandations sont additionnées. Le gain cumulé actualisé normalisé (NDCG) est le DCG divisé par le DCG idéal de sorte que le NDCG soit compris entre 0 et 1. (Le DCG idéal est celui où les K premières recommandations sont triées par pertinence). Amazon Personalize utilise un facteur de pondération de 1/log(1 + position), où le haut de la liste est la position 1. Cette métrique récompense les articles pertinents qui apparaissent près du haut de la liste, car le haut d'une liste suscite généralement plus d'attention.
- precision_at_K : Le nombre de recommandations pertinentes parmi les K premières recommandations divisé par K. Cette métrique récompense la recommandation précise des articles pertinents.

## Créer et attendre la campagne

Maintenant que vous avez une version de la solution opérationnelle, vous devez créer une campagne pour l'utiliser avec vos applications. Une campagne est une version de la solution hébergée, un point de terminaison que vous pouvez interroger pour obtenir des recommandations. La tarification est fixée en estimant la capacité de débit (demandes de personnalisation de l'utilisateur par seconde). Lors du déploiement d'une campagne, vous définissez une valeur minimale de transactions par seconde (TPS) (`minProvisionedTPS`). Ce service, comme beaucoup d'autres au sein d'AWS, évoluera automatiquement en fonction de la demande. Néanmoins, si la latence est stratégique, vous voudrez peut-être prendre des dispositions pour une demande plus importante. Pour cette démo, le seuil de débit minimum est défini à 1. Pour plus d'informations, reportez-vous à la page de [tarification](https://aws.amazon.com/personalize/pricing/).

Comme mentionné ci-dessus, la recette de personnalisation de l'utilisateur utilisée pour notre solution prend en charge l'exploration automatique des articles "cold". Vous pouvez contrôler le niveau d'exploration lors de la création de votre campagne. Le type de données `itemExplorationConfig` prend en charge les paramètres `explorationWeight` et `explorationItemAgeCutOff`. La pondération de l'exploration détermine la fréquence à laquelle les recommandations incluent les articles dont les données d'interaction ou la pertinence sont moindres. Plus la valeur est proche de 1,0, plus l'exploration est forte. À zéro, aucune exploration n'a lieu et les recommandations sont basées sur les données actuelles (pertinence). Le critère d'âge des articles à explorer détermine les articles à explorer en fonction du temps écoulé depuis la dernière interaction. Fournir l'âge maximal de l'article, en jours depuis la dernière interaction, pour définir la portée de l'exploration de l'article. Plus la valeur est élevée, plus le nombre d'articles pris en compte lors de l'exploration est élevé. Pour notre campagne ci-dessous, nous allons spécifier une pondération d'exploration de 0,5.

#### Créer une campagne

In [ ]:
def create_campaign(solution, name):
    create_campaign_response = personalize.create_campaign(
        name = "personalize-demo-" + name + '-' + suffix,
        solutionVersionArn = solution['solution_version_arn'],
        minProvisionedTPS = 1,
        campaignConfig = {
            "itemExplorationConfig": {
                "explorationWeight": "0.5"
            }
        }
    )

    campaign_arn = create_campaign_response['campaignArn']
    print('campaign_arn:' + campaign_arn)
    return campaign_arn

def waitForCampaign(solution):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = solution['campaign_arn']
        )
        status = describe_campaign_response["campaign"]["status"]
        print("Campaign: {} {}".format(solution['campaign_arn'], status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)

#### Créer trois campagnes
Créer une campagne pour chacune des optimisations des objectifs, mais garder tous les autres paramètres identiques pour démontrer l'impact de l'optimisation des objectifs.

In [ ]:
high_solution['campaign_arn'] = create_campaign(high_solution, 'high')
low_solution['campaign_arn'] = create_campaign(low_solution, 'low')
no_objective_optimization_solution['campaign_arn'] = create_campaign(no_objective_optimization_solution, 'max_relevance')


#### Attendre que la campagne ait le statut ACTIF

Cela devrait prendre environ 10 minutes.

In [ ]:
waitForCampaign(high_solution)
waitForCampaign(low_solution)
waitForCampaign(no_objective_optimization_solution)

## Obtenir des exemples de recommandations

Une fois la campagne active, vous êtes prêt à recevoir des recommandations. Tout d'abord, nous devons sélectionner un utilisateur aléatoire à partir de la collection. Ensuite, nous créerons quelques fonctions d'aide pour obtenir des informations sur les films à afficher pour les recommandations au lieu de simples ID.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE'] + '(' + f'{-1*items.iloc[movie_id]["ROYALTY"]:.2f}'+ ')'

def get_movie_royalty(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return -1*items.iloc[movie_id]["ROYALTY"]

#### Appeler GetRecommendations

En utilisant l'utilisateur que vous avez obtenu ci-dessus, les lignes ci-dessous obtiendront des recommandations pour vous et retourneront la liste des films recommandés.


In [ ]:
def get_recommendations(solution):
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = solution['campaign_arn'],
        userId = str(user_id),
    )
    # Update DF rendering
    pd.set_option('display.max_rows', 30)

    item_list = get_recommendations_response['itemList']

    recommendation_list = []

    total_royalties = 0.0
    
    for item in item_list:
        title = get_movie_title(item['itemId'])
        total_royalties = total_royalties + get_movie_royalty(item['itemId'])
        recommendation_list.append(title)
        
    recommendation_list.append('TOTAL ROYALTIES: '+ f'{total_royalties:.2f}')
    return recommendation_list

#### Comparer les recommandations
Créer un ensemble de recommandations pour le même utilisateur afin de comparer l'impact de l'optimisation des objectifs.

Remarquez l'impact de la valeur de la redevance indiquée entre parenthèses après le titre et l'année. Les films très bien notés et à forte redevance dont l'optimisation objective est désactivée ont tendance à apparaître plus bas dans la liste, voire pas du tout, lorsque l'optimisation objective est activée. 

Remarquez également les redevances totales pour tous les titres de chaque série de recommandations.

In [ ]:
recommendations_df = pd.DataFrame(get_recommendations(no_objective_optimization_solution), columns = ['ObjectiveOff'])
recommendations_df['LowObjective'] = get_recommendations(low_solution)
recommendations_df['HighObjective'] = get_recommendations(high_solution)
recommendations_df

## Examiner

Le bloc-notes présente un exemple où le moteur de recommandation de films tient compte des redevances à payer pour un titre donné. En incluant cette pondération dans l'algorithme de recommandation, le service de streaming peut fournir de bonnes recommandations à un utilisateur, mais aussi minimiser les redevances à payer aux créateurs de contenu.

Remarquez dans le graphique ci-dessus que, lorsque l'optimisation de l'objectif est désactivée, les frais de redevance entre parenthèses sont répartis assez uniformément, comme on peut s'y attendre puisqu'ils ne sont pas pris en compte. Cependant, dans la colonne LowObjective, les valeurs sont plus faibles et la somme des redevances est la plus basse pour le paramètre d'optimisation de l'objectif le plus élevé.

## Nettoyer

Nettoyer les ressources

In [ ]:
def delete_campaign(campaign_arn):
    delete_campaign_result = personalize.delete_campaign(campaignArn=campaign_arn )
    

def wait_for_delete_campaign(campaign_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_campaign_response = personalize.describe_campaign(
                campaignArn = campaign_arn
            )
            status = describe_campaign_response["campaign"]["status"]
            print("campaign: {}".format(status))

        except ClientError as e:
            print(e)
            break

        time.sleep(10)
    print('campaign ' + campaign_arn + ' deleted')
    
def delete_solution(solution_arn):
    delete_solution_result = personalize.delete_solution(solutionArn=solution_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours

def wait_for_delete_solution(solution_arn):
    while time.time() < max_time:
        
        try:
            describe_solution_response = personalize.describe_solution(
                solutionArn = solution_arn
            )
            status = describe_solution_response["solution"]["status"]
            print("Solution: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('Solution ' + solution_arn + ' deleted')
    
def delete_dataset(dataset_arn):
    delete_dataset_result = personalize.delete_dataset(datasetArn=dataset_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_response = personalize.describe_dataset(
                datasetArn = dataset_arn
            )
            status = describe_dataset_response["dataset"]["status"]
            print("dataset: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset ' + dataset_arn + ' deleted')
    
def delete_schema(schema_arn):
    delete_schema_result = personalize.delete_schema(schemaArn=schema_arn )
    

    print('schema ' + schema_arn + ' deleted')
    
def delete_dataset_group(dataset_group_arn):
    delete_dataset_group_result = personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_group_response = personalize.describe_dataset_group(
                datasetGroupArn = dataset_group_arn
            )
            status = describe_dataset_group_response["datasetGroup"]["status"]
            print("dataset_group: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset_group ' + dataset_group_arn + ' deleted')
    
def delete_all(solutions):
    for solution in solutions:
        delete_campaign(solution['campaign_arn'])
        
    for solution in solutions:
        wait_for_delete_campaign(solution['campaign_arn'])
    for solution in solutions:
        delete_solution(solution['solution_arn'])
    for solution in solutions:
        wait_for_delete_solution(solution['solution_arn'])


    

In [ ]:
delete_all([no_objective_optimization_solution, low_solution, high_solution] )

In [ ]:
delete_dataset(item_dataset_arn)
delete_dataset(interaction_dataset_arn)

In [ ]:
delete_schema(item_schema_arn)
delete_schema(schema_arn)

In [ ]:
delete_dataset_group(dataset_group_arn)

In [ ]:
iam = boto3.client("iam")

iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess')
iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
time.sleep(10) # propogation time

iam.delete_role(RoleName=role_name)

In [ ]:
! aws s3 rm --recursive s3://$bucket_name

In [ ]:
s3.delete_bucket(Bucket=bucket_name)